In [1]:
import torch

In [12]:
params = torch.tensor([[0.1], [0.9]], requires_grad = True)
X=params[0]
Y=params[1]
  
optimizer = torch.optim.Adam([params], lr=0.01)
steps = 2000
for i in range(steps):
  optimizer.zero_grad()
  f = (1.5-X+X*Y)**2+(2.25-X+X*Y**2)**2+(2.625-X+X*Y**3)**2  
  f.backward()
  optimizer.step()
  if i%100 == 0:
    print(f'At step {i+1 :4} x={str([round(params[i].item(), 3) for i in range(params.numel())]) :16}'\
          f' and the function value is {f.item(): 0.4f}.' )

At step    1 x=[0.11, 0.89]     and the function value is  13.9465.
At step  101 x=[1.202, 0.006]   and the function value is  3.2671.
At step  201 x=[1.818, 0.065]   and the function value is  0.8956.
At step  301 x=[2.151, 0.212]   and the function value is  0.3241.
At step  401 x=[2.357, 0.294]   and the function value is  0.1450.
At step  501 x=[2.496, 0.346]   and the function value is  0.0746.
At step  601 x=[2.598, 0.382]   and the function value is  0.0418.
At step  701 x=[2.676, 0.408]   and the function value is  0.0246.
At step  801 x=[2.738, 0.427]   and the function value is  0.0150.
At step  901 x=[2.787, 0.442]   and the function value is  0.0093.
At step 1001 x=[2.828, 0.454]   and the function value is  0.0058.
At step 1101 x=[2.861, 0.463]   and the function value is  0.0036.
At step 1201 x=[2.889, 0.471]   and the function value is  0.0023.
At step 1301 x=[2.911, 0.477]   and the function value is  0.0014.
At step 1401 x=[2.93, 0.482]    and the function value is  0.